<a href="https://colab.research.google.com/github/Phil-Schwarz/TQA/blob/squbr%2Fcombiner_network/combiner_network/combiner_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [2]:
!pip install jsonlines

In [7]:
from keras.layers import Dense, Input, TimeDistributed, LSTM
from keras.layers import Concatenate, Dropout, SpatialDropout1D
from keras.models import Model
import argparse
import os
import uuid

#import arch, utils
#from settings import TRAIN_DATA_PATH, VAL_DATA_PATH
#from settings import MODELS_DIR, TOP_N

from keras.callbacks import ModelCheckpoint

# Kv_attention
#import tensorflow as tf
#from keras import backend as K
#from keras.engine import Layer

#import torch
#import torch.nn as nn
#import torchvision
#import torchvision.transforms as transforms

import jsonlines
import pandas as pd

import random
import numpy as np

## CONSTANTS

In [19]:
# TRAIN_DATA_PATH =
# VAL_DATA_PATH = 
DOC_QUANTITY = 2
NUM_SCORES = 3
#LEN_DATASET = 4000
TOP_N = 20
MODELS_DIR = "content/model"

## Generate Random Input Data to train Neural Network

In [ ]:
#utils.py

# Returns a dictionary of entries (for answers) and their labels. To be
# directly wired into Keras methods (fit, predict, etc.).
# Does not shuffle the questions.
class utils:

  # Returns a list of entries. To be fed into @to_numpy to extract tensors
  # that can be directly wired to Keras @fit.
  def read_dataset(file_path, top_n, mask_IR=False,
                mask_Ranking=False, mask_answer_verifier=False):
    # assert(isinstance(top_n, int))
    # assert(top_n >= 1)

    data = None
    with open(file_path, 'r') as f:
        data = json.load(f)
    # assert(data is not None)
    # assert(len(data) == 1)
    # assert("data" in data)

    def chunks(array, n):
        """Yield successive n-sized chunks from array."""
        for i in range(0, len(array), n):
            yield array[i:i + n]

    def keep_top_n(array):
        # assert(isinstance(array, list))
        # assert(len(array) >= top_n * NUM_SOURCES)
        # assert(len(array) % NUM_SOURCES == 0)
        chunk_size = len(array) // NUM_SOURCES
        array = list(chunks(array, chunk_size))
        # assert(len(array) == NUM_SOURCES)
        # for chunk_id in range(0, NUM_SOURCES):
            # array[chunk_id] = array[chunk_id][:top_n]
        processed_array = []
        for chunk_id in range(0, NUM_SOURCES):
            # assert(len(array[chunk_id]) == top_n)
            processed_array += array[chunk_id]
        # assert(len(processed_array) == top_n * NUM_SOURCES)
        return processed_array

    data = data["data"]
    assert(isinstance(data, list))
    for entry in data:
        # assert(len(entry) == 6)
        # assert("question_id" in entry)
        # assert("answers" in entry)
        # assert("correct_answer" in entry)
        # assert("question_text" in entry)
        # assert("answers_text" in entry)
        # assert("documents" in entry)
        # assert(entry["correct_answer"] in [0, 1, 2, 3])
        # assert(isinstance(entry["answers"], list))
        # assert(len(entry["answers"]) == 4)
        # assert(isinstance(entry["documents"], list))
        # assert(len(entry["documents"]) == 4)
        # assert(isinstance(entry["question_text"], str))
        # assert(len(entry["answers_text"]) == 4)
        # for answer_text in entry["answers_text"]:
        #     assert(isinstance(answer_text, str))

        # Keep only TOP_N entries from each source.
        for idx in [0, 1, 2, 3]:
            entry["answers"][idx] = keep_top_n(entry["answers"][idx])
            # assert(len(entry["answers"][idx]) == top_n * NUM_SOURCES)
            entry["documents"][idx] = keep_top_n(entry["documents"][idx])
            # assert(len(entry["documents"][idx]) == top_n * NUM_SOURCES)

        for answer in entry["answers"]:
            # assert(isinstance(answer, list))
            # assert(len(answer) == top_n * NUM_SOURCES)
            for local_answer in answer:
                # assert(isinstance(local_answer, list))
                # assert(len(local_answer) == NUM_FEATURES)
                if mask_IR:
                    local_answer[1] = 0.0
                if mask_Ranking:
                    local_answer[2] = 0.0
                if mask_answer_verifier:
                    local_answer[3] = 0.0
            # assert([x[0] for x in answer] == NUM_SOURCES * list(range(1, top_n + 1)))  # noqa: E501
    return data


  # Augment the dataset with all answer permutations such that the neural
  # network cannot learn how to decide based on positions.
  # Returns the newly constructed dataset.
  def augment_with_permutations(dataset):
    assert(isinstance(dataset, list))
    augmented = []
    for entry in dataset:
      # assert(len(entry) == 6)
      # assert("question_id" in entry)
      # assert("answers" in entry)
      # assert("correct_answer" in entry)
      # assert("question_text" in entry)
      # assert("answers_text" in entry)
      # assert("documents" in entry)
      for answer_text in entry["answers_text"]:
          assert(isinstance(answer_text, str))

      correct_answer = entry["correct_answer"]
      answers = entry["answers"]
      documents = entry["documents"]
      answers_text = entry["answers_text"]
      # assert(correct_answer in [0, 1, 2, 3])
      # assert(isinstance(answers, list) and len(answers) == 4)
      # assert(isinstance(documents, list) and len(documents) == 4)
      for perm in itertools.permutations([0, 1, 2, 3]):
        permuted_answers = [answers[i] for i in perm]
        permuted_answers_text = [answers_text[i] for i in perm]
        permuted_documents = [documents[i] for i in perm]
        permuted_correct_answer = perm.index(correct_answer)
        # assert(permuted_correct_answer in [0, 1, 2, 3])
        augmented.append({
                "question_id": entry["question_id"],
                "correct_answer": permuted_correct_answer,
                "answers": permuted_answers,
                "question_text": entry["question_text"],
                "documents": permuted_documents,
                "answers_text": permuted_answers_text
        })
    # assert(len(augmented) == 24 * len(dataset))
    # for entry in augmented:
    #     assert(len(entry) == 6)
    #     assert("question_id" in entry)
    #     assert("answers" in entry)
    #     assert("correct_answer" in entry)
    #     assert("question_text" in entry)
    #     assert("answers_text" in entry)
    #     assert("documents" in entry)
    #     for answer_text in entry["answers_text"]:
    #         assert(isinstance(answer_text, str))
    return augmented

  def to_numpy(dataset, top_n):
    assert(isinstance(dataset, list))
    assert(isinstance(top_n, int))
    assert(top_n >= 1)

    answer_a = np.zeros((len(dataset), NUM_DSOURCES * top_n, NUM_FEATURES),
                        dtype="float")
    answer_b = np.zeros((len(dataset), NUM_SOURCES * top_n, NUM_FEATURES),
                        dtype="float")
    answer_c = np.zeros((len(dataset), NUM_SOURCES * top_n, NUM_FEATURES),
                        dtype="float")
    answer_d = np.zeros((len(dataset), NUM_SOURCES * top_n, NUM_FEATURES),
                        dtype="float")
    labels = np.zeros((len(dataset), 4), dtype="int32")

    def process_answer(answer):
      # assert(isinstance(answer, list))
      answer = np.array(answer, dtype="float")
      # assert(answer.shape == (NUM_SOURCES * top_n, NUM_FEATURES))
      answer[:, 0] *= (1.0 / top_n)
      return answer

    for (idx, entry) in enumerate(dataset):
      answers = entry["answers"]
      correct_answer = entry["correct_answer"]

      answer_a[idx] = process_answer(answers[0])
      answer_b[idx] = process_answer(answers[1])
      answer_c[idx] = process_answer(answers[2])
      answer_d[idx] = process_answer(answers[3])

      labels[idx][correct_answer] = 1

    # assert(isinstance(answer_a, np.ndarray))
    # assert(isinstance(answer_b, np.ndarray))
    # assert(isinstance(answer_c, np.ndarray))
    # assert(isinstance(answer_d, np.ndarray))
    # assert(isinstance(labels, np.ndarray))
    return {
            "answer_a": answer_a,
            "answer_b": answer_b,
            "answer_c": answer_c,
            "answer_d": answer_d
    }, labels

In [37]:
# generating random test data
def create_random_traindata(LEN_DATASET):
  answer_a = np.zeros((LEN_DATASET, DOC_QUANTITY , NUM_SCORES),
                      dtype="float")
  answer_b = np.zeros((LEN_DATASET, DOC_QUANTITY , NUM_SCORES),
                      dtype="float")
  answer_c = np.zeros((LEN_DATASET, DOC_QUANTITY , NUM_SCORES),
                      dtype="float")
  answer_d = np.zeros((LEN_DATASET, DOC_QUANTITY , NUM_SCORES),
                      dtype="float")
  labels = np.zeros((LEN_DATASET, 2,4), dtype="int32")

  for i in range(0,LEN_DATASET):
    rs1 = random.uniform(-1,1)
    rs2 = random.uniform(-1,1)
    ms1 = random.uniform(-1,1)
    ms2 = random.uniform(-1,1)
    answer_a[i][0] = np.array((rs1,ms1,ms2))
    answer_a[i][1] = np.array((rs2,ms1,ms2))

    rs1 = random.uniform(-1,1)
    rs2 = random.uniform(-1,1)
    ms1 = random.uniform(-1,1)
    ms2 = random.uniform(-1,1)
    answer_b[i][0] = np.array((rs1,ms1,ms2))
    answer_b[i][1] = np.array((rs2,ms1,ms2))

    rs1 = random.uniform(-1,1)
    rs2 = random.uniform(-1,1)
    ms1 = random.uniform(-1,1)
    ms2 = random.uniform(-1,1)
    answer_c[i][0] = np.array((rs1,ms1,ms2))
    answer_c[i][1] = np.array((rs2,ms1,ms2))

    rs1 = random.uniform(-1,1)
    rs2 = random.uniform(-1,1)
    ms1 = random.uniform(-1,1)
    ms2 = random.uniform(-1,1)
    answer_d[i][0] = np.array((rs1,ms1,ms2))
    answer_d[i][1] = np.array((rs2,ms1,ms2))

    labels[i] = np.zeros(4, dtype="int32")
    index = random.randint(0, 3)
    labels[i][0][index] = 1
    labels[i][1][index] = 1
  return {
          "answer_a": answer_a,
          "answer_b": answer_b,
          "answer_c": answer_c,
          "answer_d": answer_d
  }, labels
#create_random_traindata(4000)
answer_a.shape


TensorShape([None, 2, 3])

## Get and prepare Dataset for Input

In [ ]:
##  Get ARC Dataset from my GoogleDrive
!gdown --id 1Fp5TIgoqJLg0uL69WSWlaztN7EclxwJl 

#Unzip
!jar xvf ARC-V1-Feb2018.zip

path_easy = "ARC-V1-Feb2018-2/ARC-Easy/ARC-Easy-Train.jsonl"
path_chall = "ARC-V1-Feb2018-2/ARC-Challenge/ARC-Challenge-Train.jsonl"

def extract_questions_and_answers(path):
  data_rows = []
  answer_choices = []

  with jsonlines.open(path) as reader:
    for obj in reader:
      id = obj["id"]
      question = obj["question"]["stem"]
      answer_key = obj["answerKey"]# more possible answers
      for answer_choice in obj["question"]["choices"]:
        answer_choices.append(answer_choice["label"]+": "+answer_choice["text"])

      data_rows.append({
            "id":id,
            "question":question,
            "answer_choices":answer_choices,
            "answer_key":answer_key
      })
  return pd.DataFrame(data_rows)

extract_questions_and_answers(path_easy)

# # MNIST dataset 
# train_dataset = torchvision.datasets.MNIST(root='./data', 
#                                            train=True, 
#                                            transform=transforms.ToTensor(),  
#                                            download=True)

# test_dataset = torchvision.datasets.MNIST(root='./data', 
#                                           train=False, 
#                                           transform=transforms.ToTensor())

# # Data loader
# train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
#                                            batch_size=batch_size, 
#                                            shuffle=True)

# test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
#                                           batch_size=batch_size, 
#                                           shuffle=False)

## LSTM Layer

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
# input_size = 784 # 28x28
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.001

input_size = 3 # one row at a time (tf-idf, destilbert, albert)                 #28
sequence_length = 20 # amount of documents                                      #28
hidden_size = 128
num_layers = 2 # amount of rnn/lstm boxes

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        # -> x needs to be: (batch_size, seq, input_size)                         #what is batchsize -Batch size is a term used in machine learning
                                                                                  # and refers to the number of training examples utilized in one iteration. 
        
        # or:
        #self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        #self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        # Set initial hidden states (and cell states for LSTM)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        
        # x: (n, 28, 28), h0: (2, n, 128)
        
        # Forward propagate RNN
        #out, _ = self.rnn(x, h0)  
        # or:
        out, _ = self.lstm(x, (h0,c0))                                            # underscore because we dont need this
        
        # out: tensor of shape (batch_size, seq_length, hidden_size)
        # out: (n, 28, 128)
        
        # Decode the hidden state of the last time step
        out = out[:, -1, :]
        # out: (n, 128)
         
        out = self.fc(out)                                                        # fully connected
        # out: (n, 10)
        return out


## KVAttention Layer

In [ ]:
# first we will implement a lstm and later on (if we have time) we will try to include kv-attention

# this class is used as a layer in the model
# this will be skipped at first
# from kv_attention.py
class KVAttention(Layer):

    # attention_size is the variable k from the paper.
    def __init__(self, key_size, value_size,
                 return_attention_scores=False, **kwargs):
        if K.backend() != 'tensorflow':
            raise RuntimeError('KVAttention is only available with '
                               'the TensorFlow backend.')
        assert(isinstance(key_size, int) and key_size >= 1)
        assert(isinstance(value_size, int) and value_size >= 1)
        assert(isinstance(return_attention_scores, bool))

        self.key_size = key_size
        self.value_size = value_size
        self.return_attention_scores = return_attention_scores
        self.imput_dim = None
        self.timestamps = None

        super(KVAttention, self).__init__(**kwargs)

    # The model receives an input with shape (batch_size, timestamp, input_dim)
    def build(self, input_shape):
        assert(len(input_shape) == 3)

        self.timestamps = input_shape[1]
        self.input_dim = input_shape[2]

        self.key_embed_w = self.add_weight(
                        shape=(self.input_dim, self.key_size),
                        name='key_embed_w',
                        initializer='glorot_uniform',
                        trainable=True
        )
        self.key_embed_b = self.add_weight(
                        shape=(self.key_size,),
                        name='key_embed_b',
                        initializer='zeros',
                        trainable=True
        )
        self.key_to_scalar_w = self.add_weight(
                        shape=(self.key_size, 1),
                        name='key_to_scalar_w',
                        initializer='glorot_uniform',
                        trainable=True
        )
        self.key_to_scalar_b = self.add_weight(
                        shape=(1,),
                        name='key_to_scalar_b',
                        initializer='zeros',
                        trainable=True
        )
        self.value_embed_w = self.add_weight(
                        shape=(self.input_dim, self.value_size),
                        name='value_embed_w',
                        initializer='glorot_uniform',
                        trainable=True
        )
        self.value_embed_b = self.add_weight(
                        shape=(self.value_size,),
                        name='value_embed_b',
                        initializer='zeros',
                        trainable=True
        )

        super(KVAttention, self).build(input_shape)

    def call(self, inputs, mask=None):
        input_tensor = inputs  # (batch_size, timestamp, input_dim)
        assert(len(input_tensor.shape) == 3)
        assert(input_tensor.shape[1] == self.timestamps)
        assert(input_tensor.shape[2] == self.input_dim)

        # K = tanh(W_emb * input_tensor + b_emb).
        K = tf.reshape(input_tensor, [-1, self.input_dim])
        K = tf.nn.xw_plus_b(K, self.key_embed_w, self.key_embed_b)
        K = tf.tanh(K)  # K.shape = (batch * timestamp, key_size)

        # Further encode the key into a single scalar for each timestamp.
        K = tf.nn.xw_plus_b(K, self.key_to_scalar_w, self.key_to_scalar_b)
        K = tf.tanh(K)  # K.shape = (batch * timestamp, 1)
        K = tf.reshape(K, [-1, self.timestamps])
        # K.shape = (batch_size, timestamp)

        # Apply softmax to the Key tensor.
        assert(len(K.shape) == 2)
        assert(K.shape[1] == self.timestamps)
        P = tf.nn.softmax(K)  # P.shape (batch_size, timestamps)
        assert(P.shape[1:] == K.shape[1:])
        if self.return_attention_scores:
            return P

        # Build the Value vector (key part is completed, we have P).
        # V = tanh(W_emb2 * input_tensor + b_emb2).
        V = tf.reshape(input_tensor, [-1, self.input_dim])
        V = tf.nn.xw_plus_b(V, self.value_embed_w, self.value_embed_b)
        V = tf.nn.relu(V)  # V.shape = (batch * timestamp, value_size)
        V = tf.reshape(V, [-1, self.timestamps, self.value_size])
        # V.shape = (batch_size, timestamp, value_size)

        # Perform the weighted sum.
        P = tf.expand_dims(P, 1)
        # P.shape = (batch_size, 1, timestamps)
        # V.shape = (batch_size, timestamps, value_size)

        out = tf.matmul(P, V)
        assert(len(out.shape) == 3)
        assert(out.shape[1] == 1)
        out = tf.squeeze(out, axis=1)
        assert(out.shape[1] == self.value_size)

        return out

    def compute_output_shape(self, input_shape):
        assert(isinstance(input_shape, tuple))
        assert(len(input_shape) == 3)
        if self.return_attention_scores:
            return (input_shape[0], self.timestamps)
        return (input_shape[0], self.value_size)

## Arch.py

In [38]:
# from arch.py

def get_model():
  answer_a = Input(shape=(DOC_QUANTITY , NUM_SCORES), name="answer_a") 
  answer_b = Input(shape=(DOC_QUANTITY , NUM_SCORES), name="answer_b") 
  answer_c = Input(shape=(DOC_QUANTITY , NUM_SCORES), name="answer_c") 
  answer_d = Input(shape=(DOC_QUANTITY , NUM_SCORES), name="answer_d") 
  # These layers are shared for each answer.
  # apply a layer to every temporal slice of an input
  #  The input should be at least 3D, and the dimension of index one
  #  will be considered to be the temporal dimension.
  encoder_layer1 = TimeDistributed(
                      Dense(32, activation='tanh', name="dense_1"),
                      name="time_distributed_1"
  )
  dropout_layer1 = SpatialDropout1D(0.25, name="spatial_dropout_1")
  # implementing lstm
  # input_dim = 3
  hidden_dim = 32
  # n_layers = 100
  lstm_layer = LSTM(hidden_dim, return_sequences=True, dropout = 0.15) #do it in keras!!!
  # to_scalar_layer = KVAttention(
  #                     key_size=64, value_size=8, name="att",
  #                     return_attention_scores=return_attention_scores
  # )
  def encode_answer(answer):
    x = encoder_layer1(answer)
    x = dropout_layer1(x)
    x = lstm_layer(x)
  # x = to_scalar_layer(x)
    return x
  a = encode_answer(answer_a)
  b = encode_answer(answer_b)
  c = encode_answer(answer_c)
  d = encode_answer(answer_d)
  y = Concatenate(axis=-1, name="concatenate_answers")([a, b, c, d])
  # if return_attention_scores:
  #   output = y
  # else:
  y = Dense(32, activation='relu', name="dense_2")(y)
  y = Dropout(0.1, name="dropout_1")(y)
  y = Dense(32, activation='relu', name="dense_3")(y)
  y = Dropout(0.1, name="dropout_2")(y)
  y = Dense(32, activation='relu', name="dense_4")(y)
  print(y.shape)
  output = Dense(4, activation='softmax', name="dense_5")(y)
  print(output.shape)

  model = Model(inputs=[answer_a, answer_b, answer_c, answer_d],
                    outputs=[output])
  return model
get_model()

(None, 2, 32)
(None, 2, 4)


## Setup Model Train

In [39]:
# from train.py
def main():
    # parser = argparse.ArgumentParser()
    # parser.add_argument('-n', '--top_n', type=int,
    #                     required=False, default=TOP_N,
    #                     help='Number of documents to use.')
    # FLAGS, _ = parser.parse_known_args()
    # top_n = FLAGS.top_n

    # train_dataset = utils.read_dataset(TRAIN_DATA_PATH, top_n)
    # train_dataset = utils.augment_with_permutations(train_dataset)
    # train_data, train_labels = utils.to_numpy(train_dataset, top_n)
    train_data, train_labels = create_random_traindata(4000)
    # del train_dataset

    # val_dataset = utils.read_dataset(VAL_DATA_PATH, top_n)
    # val_data, val_labels = utils.to_numpy(val_dataset, top_n)
    val_data, val_labels = create_random_traindata(700)
    # del val_dataset

    model = get_model()
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()

    filepath = os.path.join(
                    MODELS_DIR,
                    "model-" + str(uuid.uuid4()) +
                    "-{epoch:04d}-{val_loss:.4f}-{val_acc:.4f}.hdf5"
    )
    save_model = ModelCheckpoint(filepath, monitor='val_acc', verbose=0,
                                 save_best_only=False, mode='max')
    model.fit(
            train_data, train_labels,
            validation_data=(val_data, val_labels),
            batch_size=128,
            epochs=75,
            verbose=1,
            callbacks=[save_model]
    )
#execute    
main()

(None, 2, 32)
(None, 2, 4)
Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
answer_a (InputLayer)           [(None, 2, 3)]       0                                            
__________________________________________________________________________________________________
answer_b (InputLayer)           [(None, 2, 3)]       0                                            
__________________________________________________________________________________________________
answer_c (InputLayer)           [(None, 2, 3)]       0                                            
__________________________________________________________________________________________________
answer_d (InputLayer)           [(None, 2, 3)]       0                                            
_________________________________________________________________

## Run trained Model on Validaion Set

In [40]:
def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('-n', '--top_n', type=int,
                        required=False, default=TOP_N,
                        help='Number of documents to use.')
    FLAGS, _ = parser.parse_known_args()
    top_n = FLAGS.top_n

    train_dataset = utils.read_dataset(TRAIN_DATA_PATH, top_n)
    train_dataset = utils.augment_with_permutations(train_dataset)
    train_data, train_labels = utils.to_numpy(train_dataset, top_n)
    del train_dataset

    val_dataset = utils.read_dataset(VAL_DATA_PATH, top_n)
    val_data, val_labels = utils.to_numpy(val_dataset, top_n)
    del val_dataset

    model = arch.get_model(top_n)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()

    filepath = os.path.join(
                    MODELS_DIR,
                    "model-" + str(uuid.uuid4()) +
                    "-{epoch:04d}-{val_loss:.4f}-{val_acc:.4f}.hdf5"
    )
    save_model = ModelCheckpoint(filepath, monitor='val_acc', verbose=0,
                                 save_best_only=False, mode='max')
    model.fit(
            train_data, train_labels,
            validation_data=(val_data, val_labels),
            batch_size=128,
            epochs=75,
            verbose=1,
            callbacks=[save_model]
    )